In [17]:
import pandas as pd
import sqlite3




## building database for Flask app

In [22]:
df = pd.read_csv("../static/data/slc_daily_pm2.5_pm10_2016_present.csv")
df = df.sort_values(['date'], ascending = False)

# code for just latest
#latest_sensor_df = df.groupby(['sensor_id']).first().reset_index()
#latest_sensor_df = latest_sensor_df[['sensor_id','latitude','longitude','pm2.5_atm']]

,sensor_id,pm2.5_atm,pm10.0_atm,date,latitude,longitude,date_created,model,hardware,position_rating,altitude
268877,165997,2.2205,2.4150,9/9/23,40.733875,-111.841090,1665080335,PA-II,2.0+BME280+PMSX003-B+PMSX003-A,5,4561
269301,169779,2.2595,2.8850,9/9/23,40.769947,-111.847015,1667930031,PA-II-FLEX,3.0+RV3028+BME68X+PMSX003-A+PMSX003-B,5,4758
231900,91975,3.0910,4.2670,9/9/23,40.710438,-111.847490,1606170343,PA-II,2.0+BME280+PMSX003-B+PMSX003-A,5,4455
66556,6622,2.6005,2.7985,9/9/23,40.743610,-111.876320,1517265414,PA-II-SD,2.0+OPENLOG+NO-DISK+DS3231+BME280+PMSX003-B+PM...,5,4256
167519,33077,1.8840,2.0530,9/9/23,41.137455,-112.029526,1560273739,PA-II,2.0+BME280+PMSX003-B+PMSX003-A,5,4523
...,...,...,...,...,...,...,...,...,...,...,...
269807,49155,0.8900,1.0600,04/01/2024,40.735126,-111.882286,1582736429,PA-II-SD,2.0+OPENLOG+15802 MB+DS3231+BME280+PMSX003-B+P...,5,4265
269806,49061,1.9450,1.9650,04/01/2024,40.748310,-111.840210,1582656580,PA-II,2.0+BME280+PMSX003-B+PMSX003-A,5,4652
269805,47171,3.5300,3.7250,04/01/2024,40.689796,-112.082690,1580166352,PA-II,2.0+BME280+PMSX003-B+PMSX003-A,5,4473
269804,46881,0.5700,0.6200,04/01/2024,40.675575,-112.019610,1579894473,PA-II,2.0+BME280+PMSX003-B+PMSX003-A,5,4489


In [24]:
# convert date
def parse_date(date_str):
    for fmt in ('%m/%d/%Y', '%m/%d/%y'):  #formats to try
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError: 
            continue
    return pd.NaT


df['date'] = df['date'].apply(parse_date)



        sensor_id  pm2.5_atm  pm10.0_atm       date   latitude   longitude  \
268877     165997     2.2205      2.4150 2023-09-09  40.733875 -111.841090   
269301     169779     2.2595      2.8850 2023-09-09  40.769947 -111.847015   
231900      91975     3.0910      4.2670 2023-09-09  40.710438 -111.847490   
66556        6622     2.6005      2.7985 2023-09-09  40.743610 -111.876320   
167519      33077     1.8840      2.0530 2023-09-09  41.137455 -112.029526   
...           ...        ...         ...        ...        ...         ...   
269807      49155     0.8900      1.0600 2024-04-01  40.735126 -111.882286   
269806      49061     1.9450      1.9650 2024-04-01  40.748310 -111.840210   
269805      47171     3.5300      3.7250 2024-04-01  40.689796 -112.082690   
269804      46881     0.5700      0.6200 2024-04-01  40.675575 -112.019610   
269824      91975     2.2800      4.4500 2024-04-01  40.710438 -111.847490   

        date_created       model  \
268877    1665080335       

In [25]:
# examine missing values for altitude
df_sensors = df[['sensor_id','latitude','longitude','date','altitude','pm2.5_atm','pm10.0_atm']]
df_sensors.rename(columns = {'pm2.5_atm':'pm2','pm10.0_atm':'pm10'},inplace=True)
df_sensors['date'] = pd.to_datetime(df['date'], format='%m/%d/%y')
sum(df_sensors['altitude'].isna())

/var/folders/kn/v0dr_8xn1qj15xgwlhdyfzph0000gn/T/ipykernel_17333/1837659829.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sensors.rename(columns = {'pm2.5_atm':'pm2','pm10.0_atm':'pm10'},inplace=True)
/var/folders/kn/v0dr_8xn1qj15xgwlhdyfzph0000gn/T/ipykernel_17333/1837659829.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sensors['date'] = pd.to_datetime(df['date'], format='%m/%d/%y')


0

In [26]:
# SQLite database name
database_name = '../static/data/sensors_readings_2016_present.db'

# create it if it doesn't exist
conn = sqlite3.connect(database_name)

# Convert the DataFrame to a SQL table within the database
# Replace 'data_table' with your desired table name
# Correct usage with a string table name
df_sensors.to_sql('sensors_readings', conn, if_exists='replace', index=False)


# Close the connection to the database
conn.close()

In [27]:
df_sensors

,sensor_id,latitude,longitude,date,altitude,pm2,pm10
268877,165997,40.733875,-111.841090,2023-09-09,4561,2.2205,2.4150
269301,169779,40.769947,-111.847015,2023-09-09,4758,2.2595,2.8850
231900,91975,40.710438,-111.847490,2023-09-09,4455,3.0910,4.2670
66556,6622,40.743610,-111.876320,2023-09-09,4256,2.6005,2.7985
167519,33077,41.137455,-112.029526,2023-09-09,4523,1.8840,2.0530
...,...,...,...,...,...,...,...
269807,49155,40.735126,-111.882286,2024-04-01,4265,0.8900,1.0600
269806,49061,40.748310,-111.840210,2024-04-01,4652,1.9450,1.9650
269805,47171,40.689796,-112.082690,2024-04-01,4473,3.5300,3.7250
269804,46881,40.675575,-112.019610,2024-04-01,4489,0.5700,0.6200
